In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('/home/kazuki_onodera/Python')
import xgbextension as ex

In [2]:
train = pd.read_pickle('../data/train2.p')
test = pd.read_pickle('../data/test2.p')

In [3]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation\nwhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,daww he matches this background colour im see...,0,0,0,0,0,0
2,000113f07ec002fd,hey man im really not trying to edit war i...,0,0,0,0,0,0
3,0001b41b1c6bb37e,\nmore\ni cant make any real suggestions on im...,0,0,0,0,0,0
4,0001d958c54c6e35,you sir are my hero any chance you remem...,0,0,0,0,0,0


In [4]:
train.drop('comment_text', axis=1, inplace=True)

In [18]:
label_names = [c for c in train.columns[1:]]

In [20]:
y_train = train[['id'] + label_names]

In [8]:
X_train = pd.merge(train[['id']], pd.read_pickle('../data/103_train_0.p'), on='id', how='left')

In [10]:
X_train.head()

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_290,vec_291,vec_292,vec_293,vec_294,vec_295,vec_296,vec_297,vec_298,vec_299
0,0000997932d777bf,0.233398,-0.041504,0.097534,0.457764,-0.225189,-0.743652,0.111328,-0.253418,-0.014648,...,0.236084,0.617432,-0.265381,0.604004,0.005981,0.082947,-0.464355,-1.356445,1.021484,-0.134338
1,000103f0d9cfb60f,0.257706,0.397949,-0.317749,0.859863,-0.253784,-0.091431,0.610474,-0.784180,0.344727,...,-0.161621,0.294464,0.086792,-0.220703,-0.157959,-0.791260,-0.419189,0.084351,0.499756,-0.085571
2,000113f07ec002fd,0.179199,-0.162109,0.305420,-0.053360,-0.105118,0.382324,0.139404,-0.054199,-0.009766,...,-0.243408,0.440125,-0.207031,-0.542969,-0.411133,-0.354248,0.220093,-0.442139,0.202881,-0.267334
3,0001b41b1c6bb37e,-0.072830,-0.138611,0.049805,0.193970,-0.083252,0.205017,0.113098,0.112305,0.455933,...,0.273926,0.167603,-0.326660,0.262939,-0.007263,0.288086,-0.380859,-0.019043,0.079159,0.056885
4,0001d958c54c6e35,0.464355,0.172363,0.768555,1.128906,-0.363037,0.092651,0.144165,-0.142212,0.166260,...,-0.380615,0.038574,-0.470459,-0.338989,-0.076294,-0.059448,-0.397827,-0.448059,0.071777,0.063660


In [13]:
param = {'colsample_bytree': 0.5,
                 'eta': 0.1,
                 'eval_metric': 'auc',
                 'max_depth': 5,
                 'objective': 'binary:logistic',
                 'silent': 1,
                 'tree_method': 'hist'}

In [22]:
yhat, imp, ret = ex.stacking(X_train.drop('id', axis=1), y_train.toxic, param, 9999, nfold=5, esr=10, feval=None, 
                             maximize=False, seed=None, loop=1, is_report=False, by=None, 
                             test=None, dtest=None, colsample=None, is_ave=True, verbose_eval=10)

param:
y/all: 15294/159571(9.58%)
dbuild row: 127656 dvalid row: 31915
building... 1/5 of 1/1
[0]	build-auc:0.728491	valid-auc:0.723307
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 10 rounds.
[10]	build-auc:0.792987	valid-auc:0.784668
[20]	build-auc:0.808848	valid-auc:0.796931
[30]	build-auc:0.825283	valid-auc:0.806258
[40]	build-auc:0.838764	valid-auc:0.814597
[50]	build-auc:0.849724	valid-auc:0.82034
[60]	build-auc:0.858816	valid-auc:0.823794
[70]	build-auc:0.866538	valid-auc:0.826689
[80]	build-auc:0.873316	valid-auc:0.828317
[90]	build-auc:0.879335	valid-auc:0.829997
[100]	build-auc:0.885012	valid-auc:0.831214
[110]	build-auc:0.890636	valid-auc:0.832151
[120]	build-auc:0.895228	valid-auc:0.832527
[130]	build-auc:0.899632	valid-auc:0.833074
[140]	build-auc:0.903939	valid-auc:0.833495
[150]	build-auc:0.907764	valid-auc:0.834063
[160]	build-auc:0.911397	valid-auc:0.834122
[170]	build-auc:0.914994	va

In [21]:
y_train

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,0,0,0,0,0,0
1,000103f0d9cfb60f,0,0,0,0,0,0
2,000113f07ec002fd,0,0,0,0,0,0
3,0001b41b1c6bb37e,0,0,0,0,0,0
4,0001d958c54c6e35,0,0,0,0,0,0
5,00025465d4725e87,0,0,0,0,0,0
6,0002bcb3da6cb337,1,1,1,0,1,0
7,00031b1e95af7921,0,0,0,0,0,0
8,00037261f536c51d,0,0,0,0,0,0
9,00040093b2687caa,0,0,0,0,0,0


In [17]:
label_names

['severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']